<a href="https://colab.research.google.com/github/Alinaderi1378/Recomendation_film/blob/main/recommendation_film.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas scikit-learn nltk hazm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 17.9 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=5e1b8addbd668e5c0b604f5f9f76a296b269c5ba4820a9eed6959e98b5eceaf9
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
    

In [6]:
# برای محیط دسکتاپ یا Google Colab
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import os

!pip install hazm
try:
    from hazm import stopwords_list, Normalizer
except ImportError:
    print("نصب hazm...")
    os.system("pip install hazm")
    from hazm import stopwords_list, Normalizer
# دانلود منابع لازم
nltk.download('punkt')

# خواندن داده‌ها
df = pd.read_csv('movies_farsi.csv', encoding='utf-8')
df['overview'] = df['overview'].fillna('')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:


# پیش‌پردازش متن فارسی
normalizer = Normalizer()
stop_words = set(stopwords_list())

def preprocess(text):
    text = normalizer.normalize(text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    return ' '.join(tokens)

df['overview_clean'] = df['overview'].apply(preprocess)

# بردارسازی با TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['overview_clean'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# تابع پیشنهاد فیلم
def get_recommendations(movie_title, top_n=3):
    if movie_title not in df['title'].values:
        return [f"فیلم «{movie_title}» یافت نشد."]
    idx = df[df['title'] == movie_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    return [f"{df.iloc[i]['title']} (شباهت: {score:.2f})" for i, score in sim_scores]


print("🎬 لیست فیلم‌های موجود:")
for idx, title in enumerate(df['title']):
    print(f"{idx + 1}. {title}")

movie_index = int(input("\nشماره فیلم مورد نظر را وارد کنید: ")) - 1

if 0 <= movie_index < len(df):
    selected_title = df.iloc[movie_index]['title']
    print(f"\n✅ فیلم انتخاب‌شده: «{selected_title}»")
    print("🎯 پیشنهاد فیلم‌های مشابه:\n")
    recommendations = get_recommendations(selected_title)
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
else:
    print("شماره وارد شده نامعتبر است.")



🎬 لیست فیلم‌های موجود:
1. جدایی نادر از سیمین
2. ابد و یک روز
3. درباره الی
4. طلا
5. متری شیش و نیم
6. 2متری شیش و نیم

شماره فیلم مورد نظر را وارد کنید: 5

✅ فیلم انتخاب‌شده: «متری شیش و نیم»
🎯 پیشنهاد فیلم‌های مشابه:

1. 2متری شیش و نیم (شباهت: 1.00)
2. درباره الی (شباهت: 0.04)
3. جدایی نادر از سیمین (شباهت: 0.04)
